In [1]:
import transformers 
import datasets
import torch
import logging

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# squad = datasets.load_dataset("squad",split="train[:6000]")
# squad = datasets.load_dataset("squad")


Found cached dataset squad (C:/Users/dama_/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 100.00it/s]


In [5]:
# torch.save(squad,"../datasets/squad.pt")

In [36]:
# squad = squad.train_test_split(test_size=0.2)

In [3]:
squad=torch.load("../datasets/squad.pt")

In [16]:
squad['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [17]:
# Preprocessing
# sep_token = '<sep>'
dataset_name = "squad"
model_type="roberta"
model_name= "roberta-base"
models_dir = "saved_models/roberta-base_mod"
checkpoint = 'roberta-base'
max_input_length = 386


# ## Training
learning_rate = 3e-5
num_epochs = 3

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [5]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_input_length ,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [6]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Loading cached processed dataset at C:\Users\dama_\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-99cdb0ee8d207e3c.arrow
Loading cached processed dataset at C:\Users\dama_\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-0aba9f6d7b63a394.arrow


In [7]:
data_collator = transformers.DefaultDataCollator()

In [8]:
model = transformers.AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use 

In [9]:
training_args = transformers.TrainingArguments(
    output_dir=models_dir,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

e:\HOGENT\2022_2023\BA\BP_Info_Support\bert\saved_models/roberta-base_mod is already a clone of https://huggingface.co/damapika/roberta-base_mod. Make sure you pull the latest changes with `repo.git_pull()`.


In [49]:
torch.cuda.empty_cache()

In [48]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 13           |        cudaMalloc retries: 16        |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   7413 MiB |   7431 MiB |  51658 MiB |  44244 MiB |\n|       from large pool |   7404 MiB |   7422 MiB |  51639 MiB |  44234 MiB |\n|       from small pool |      8 MiB |      9 MiB |     19 MiB |     10 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   7413 MiB |   7431 MiB |  51658 MiB |  44244 MiB |\n|       from large pool |   7404 MiB |   7422 MiB |

In [10]:
trainer.train()

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: damapika. Use `wandb login --relogin` to force relogin


  2%|▏         | 500/32850 [12:27<20:17:42,  2.26s/it]

{'loss': 1.936, 'learning_rate': 2.954337899543379e-05, 'epoch': 0.05}


  3%|▎         | 1000/32850 [29:58<20:20:22,  2.30s/it]

{'loss': 1.2617, 'learning_rate': 2.908675799086758e-05, 'epoch': 0.09}


...

 99%|█████████▉| 32500/32850 [12:51:29<10:18,  1.77s/it]  

{'loss': 0.4868, 'learning_rate': 3.19634703196347e-07, 'epoch': 2.97}


                                                          
100%|██████████| 32850/32850 [13:13:23<00:00,  1.45s/it]

{'eval_loss': 0.9988527894020081, 'eval_runtime': 663.1653, 'eval_samples_per_second': 15.939, 'eval_steps_per_second': 1.993, 'epoch': 3.0}
{'train_runtime': 47606.0335, 'train_samples_per_second': 5.52, 'train_steps_per_second': 0.69, 'train_loss': 0.7471855128736801, 'epoch': 3.0}


TrainOutput(global_step=32850, training_loss=0.7471855128736801, metrics={'train_runtime': 47606.0335, 'train_samples_per_second': 5.52, 'train_steps_per_second': 0.69, 'train_loss': 0.7471855128736801, 'epoch': 3.0})

In [11]:
trainer.push_to_hub()

Upload file pytorch_model.bin: 488MB [01:14, 7.91MB/s]                            To https://huggingface.co/damapika/roberta-base_mod
   43fc707..aa8e7b3  main -> main

Upload file pytorch_model.bin: 100%|██████████| 473M/473M [01:16<00:00, 6.52MB/s]
Upload file runs/Apr22_12-18-54_Damapika/events.out.tfevents.1682158739.Damapika.29852.0: 100%|██████████| 15.1k/15.1k [01:16<00:00, 204B/s]  
To https://huggingface.co/damapika/roberta-base_mod
   aa8e7b3..7e353d1  main -> main



'https://huggingface.co/damapika/roberta-base_mod/commit/aa8e7b34cb104dae626809a49106b232f0ce0d69'

In [19]:
question = "Who is Fyodor Dostoevsky?"
context = "In the world of literature, there have been many authors who have gained a reputation for their ability to create complex characters. One such author is Fyodor Dostoevsky, a Russian novelist who wrote several influential works in the 19th century."

In [20]:
question_answerer = transformers.pipeline("question-answering", model="damapika/roberta-base_mod")
question_answerer(question=question, context=context)

{'score': 0.9061723351478577,
 'start': 172,
 'end': 190,
 'answer': 'a Russian novelist'}